In [2]:
# all imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm


Matplotlib is building the font cache; this may take a moment.


In [16]:
# part 2 
# i. matching volatility 
# ii. mismatched volatility 

def bs_call_price(S, K, r, sigma, tau):
    """
    Calculate and return the Black-Scholes price of a European call option. 
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    d2 = (np.log(S / K) + (r - 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return S * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)

def bs_delta_call(S, K, r, sigma, tau):
    """
    Calculate and return the Black-Scholes delta of a European call option.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return norm.cdf(d1)


def simulate_gbm_path(S0, r, sigma_real, T, n_steps, rng):
    """
    Simulate and return stock price path following a Geometric Brownian Motion
    """
    dt = T / n_steps
    t = np.linspace(0.0, T, n_steps + 1)
    dW = np.sqrt(dt) * rng.standard_normal(n_steps)

    # Brownian motion path 
    W = np.zeros(n_steps + 1)
    W[1:] = np.cumsum(dW)

    # Drift term
    drift = (r - 0.5 * sigma_real**2) * t

    # Explicit GBM solution
    S = S0 * np.exp(drift + sigma_real * W)

    return S
    
def hedge_short_call_one_path(S_path, K, r, sigma_delta, T, hedge_every_steps):
    """
    Perform discrete-time delta hedging of a short European call option along a simulated stock price path. 
    Return final hedging Profit & Loss.   
    """
    n_steps = len(S_path) - 1
    dt = T / n_steps

    # t=0: sell call & buy delta shares
    C0 = bs_call_price(S_path[0], K, r, sigma_delta, T)
    delta = bs_delta_call(S_path[0], K, r, sigma_delta, T)
    shares = delta
    cash = C0 - shares * S_path[0]

    for i in range(n_steps):
        cash *= np.exp(r * dt)
        S_next = S_path[i+1]
        tau_next = max(T - (i+1) * dt, 0.0)

        if (i + 1) % hedge_every_steps == 0 and tau_next > 0:
            new_delta = bs_delta_call(S_next, K, r, sigma_delta, tau_next)
            cash -= (new_delta - shares) * S_next
            shares = new_delta

    S_T = S_path[-1]
    payoff = max(S_T - K, 0.0)
    portfolio_T = shares * S_T + cash
    return portfolio_T - payoff


def run_hedge_experiment(
    sigma_real,
    sigma_delta,
    hedge_freqs=(1, 2, 5),
    S0=100.0,
    K=99.0,
    r=0.06,
    T=1.0,
    n_steps=252,
    n_paths=5000,
    seed=42,):
    """
    Run a Monte Carlot delta-hedging experiment in order to solve task 1 & task 2.
    Returns summary statistics in a DataFrame. 
    """
    rng = np.random.default_rng(seed)
    all_results = {}

    for freq in hedge_freqs:
        pnls = np.empty(n_paths)
        for p in range(n_paths):
            S_path = simulate_gbm_path(S0, r, sigma_real, T, n_steps, rng)
            pnls[p] = hedge_short_call_one_path(S_path, K, r, sigma_delta, T, freq)
        all_results[freq] = pnls

    # summary statistics 
    rows = []
    for freq in hedge_freqs:
        pnls = all_results[freq]
        rows.append({
            "sigma_real": sigma_real,
            "sigma_delta": sigma_delta,
            "hedge_every_days": freq,
            "mean_pnl": pnls.mean(),
            "std_pnl": pnls.std(ddof=1),
        })
    summary_df = pd.DataFrame(rows).sort_values(["hedge_every_days"])
    return all_results, summary_df

# Task 1
task1_results, task1_summary = run_hedge_experiment(
    sigma_real=0.20,
    sigma_delta=0.20,
    hedge_freqs=(1, 2, 5),
    n_paths=3000,   
    seed=1,
)
print("\ntask 1 summary:")
print(task1_summary.to_string(index=False))

# Task 2
sigma_delta_grid = [0.10, 0.15, 0.25, 0.30] 
sigma_real_fixed = 0.20

# hedge daily, twice a week, weekly 
hedge_freqs = (1, 2, 5)

task2_summaries = []
for sigma_delta in sigma_delta_grid:
    results, summary = run_hedge_experiment(
        sigma_real=sigma_real_fixed,
        sigma_delta=sigma_delta,
        hedge_freqs=(1,2,5),
        n_paths=3000,
        seed=1
    )
    task2_summaries.append(summary)

task2_summary = pd.concat(task2_summaries, ignore_index=True)
print("\ntask 2 summary")
print(task2_summary.to_string(index=False))

#  TODO 
#  for report add plots? not asked specifically though 
#  iii. Pricing and Hedging with Implied Volatility


task 1 summary:
 sigma_real  sigma_delta  hedge_every_days  mean_pnl  std_pnl
        0.2          0.2                 1  0.009248 0.429580
        0.2          0.2                 2  0.005609 0.596369
        0.2          0.2                 5  0.030772 0.926771

task 2 summary
 sigma_real  sigma_delta  hedge_every_days  mean_pnl  std_pnl
        0.2         0.10                 1 -3.571205 2.095723
        0.2         0.10                 2 -3.553886 2.154404
        0.2         0.10                 5 -3.561096 2.214156
        0.2         0.15                 1 -1.849459 0.965016
        0.2         0.15                 2 -1.853284 1.071150
        0.2         0.15                 5 -1.843516 1.244958
        0.2         0.25                 1  1.928739 0.798826
        0.2         0.25                 2  1.929742 0.893118
        0.2         0.25                 5  1.966521 1.166031
        0.2         0.30                 1  3.878269 1.267996
        0.2         0.30             